在一個句子中尋找一組 Special token。

In [2]:
import re

sentence = "How does the shhhhi sed command edit a file? It creates a tmp file /etc/sedQhw17q to store your input first. Read /etc/nsswitch.conf file. Try 111 find 5.5.5.5:33."
regex = "/etc/sed.*"
regex_list = ["\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+", "\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", ":\d+"]

def find_spacial_token_regex(regex: str|list, sentence:str) -> str:
    ''' Return the matched 'word' if regex find it. If not found return None'''
    for word in sentence.split():
        if isinstance(regex, str):
            isMatch = re.match(regex, word)
            if isMatch:
                # print("Match :", word, 'with regex', regex)
                return word
        elif isinstance(regex, list):
            for r in regex:
                if re.match(r, word):
                    # print("Match :", word, 'with regex', r)
                    return word
        else:
            raise TypeError("regex must be str or list.")
    return None

find_spacial_token_regex(regex, sentence)

'/etc/sedQhw17q'

在一個句子中尋找許多組 Special token。

In [24]:
find_spacial_token_regex(regex_list, sentence)

Match : 5.5.5.5:33 with regex \d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+


'5.5.5.5:33'

In [6]:
class RegexMatchResult:
    def __init__(self, word: str, type: str=None, match_regex: str=None) -> None:
        self.word = word
        self.type = type
        self.match_regex = match_regex
        pass
    def __str__(self):
        return f"<RegexMatchResult word={self.word}, type={self.type}, match_regex={self.match_regex}>"
    def __repr__(self):
        return f"<RegexMatchResult word={self.word}, type={self.type}, match_regex={self.match_regex}>"

class RegexMaster:
    def __init__(self) -> None:
        self.regex_file = {
            "sed command": "bin/sed", 
            "startup": ["/etc/rc", "/etc/init.d/"],
            "process_info": "/proc/", 
            "sed temp file": "/etc/sed", 
            "selinx": "/selinux", 
            "sys": ["/sys/", "bin/", "lsb-release"], 
            "uname":"uname(.)($)",
            "dns": ["mtab", "nsswitch.conf", "resolv.conf", "/hosts"],
        }
        self.regex_process = {"command": ["^sh$", "^sed$"]} # must exact match
        self.regex_net = {
            "net address":["\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+", "\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}"], #, ":\d+" 
            "NIC": "eth[0-9]$"}
        self.regex_mem = {"Memory Address": "0x[0-9a-zA-Z]{8}"}
        self.regex_other = {"permission":"Permission.*", "ID":["UID", "GID"]}
        # 整理 regex rules
        self.all_regex_dict = {**self.regex_file,  **self.regex_process, **self.regex_net, **self.regex_mem, **self.regex_other}
        self.all_regex_list = []
        for v in self.all_regex_dict.values():
            if isinstance(v,list):
                for i in v:
                    self.all_regex_list.append(i)
            else:
                self.all_regex_list.append(v)
        pass

    def get_all_regex(self) -> list:
        return self.all_regex_list

    def find_spacial_token(self, sentence:str) -> RegexMatchResult:
        if not isinstance(sentence, str):
            raise TypeError("regex must be str or list.")
        result_list = []
        for word in sentence.split():
            for r in self.all_regex_list:
                # print('r', r)
                if re.search(r, word):
                    # print("Match :", word, 'with regex', r)
                    result_list.append(RegexMatchResult(word, match_regex=r)) # 暫時忽略 type
        if len(result_list):
            return result_list     
        return None

In [3]:
# testcase
regexMaster = RegexMaster()
print(regexMaster.get_all_regex())
result = regexMaster.find_spacial_token(sentence)
result

['.*bin/sed', '/etc/rc.*', '/etc/init.d/.*', '/proc/.*', '/etc/sed.*', '.*/selinux.*', '/sys/.*', '.*bin/*', '.*lsb-release.*', 'uname', '.*mtab.*', '.*nsswitch.conf.*', '.*resolv.conf.*', '.*/hosts.*', '^sh$', '^sed$', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', ':\\d+', 'eth[0-9]*', '0x[0-9a-zA-Z]{8}', 'Permission.*', 'UID', 'GID']


[<RegexMatchResult word=sed, type=None, match_regex=^sed$>,
 <RegexMatchResult word=/etc/sedQhw17q, type=None, match_regex=/etc/sed.*>,
 <RegexMatchResult word=/etc/nsswitch.conf, type=None, match_regex=.*nsswitch.conf.*>,
 <RegexMatchResult word=5.5.5.5:33., type=None, match_regex=\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+>,
 <RegexMatchResult word=5.5.5.5:33., type=None, match_regex=\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}>]

### 注意 re.search 和 re.match 用法不同!

In [4]:
res = re.search("nsswitch.conf", 'Read /etc/nsswitch.conf aaaa')
res

<re.Match object; span=(10, 23), match='nsswitch.conf'>

In [5]:
res = re.match(".*nsswitch.conf.*", 'Read /etc/nsswitch.conf aaaa')
res

<re.Match object; span=(0, 28), match='Read /etc/nsswitch.conf aaaa'>

In [7]:
res = re.match("uname", "'uname'")
res

In [11]:
res = re.search("\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+", "The ip address is 11.22.33.44:999.")
res

<re.Match object; span=(18, 33), match='11.22.33.44:999'>

In [14]:

res = re.search(r"uname(.)($)", "'uname'")
print(res)

res = re.search(r"uname(.)($)", "'unamefigure")
print(res)

<re.Match object; span=(1, 7), match="uname'">
None


### 文件中的句子
unamefigure

it adds the following processes:
/bin/uname

if the ‘uname’ call

processor
/sys/devices/system/cpu

by reading files /sys/class/dmi/id/product_name and

In [ ]:
# match ver
# 前後要加 .*
self.regex_file = {
    "sed command": ".*bin/sed", 
    "startup": ["/etc/rc.*", "/etc/init.d/.*"],
    "process_info": "/proc/.*", 
    "sed temp file": "/etc/sed.*", 
    "selinx": ".*/selinux.*", 
    "sys": ["/sys/.*", ".*bin/*", ".*lsb-release.*"], 
    "uname":".*uname.*",
    "dns": [".*mtab.*", ".*nsswitch.conf.*", ".*resolv.conf.*", ".*/hosts.*"],
}
self.regex_process = {"command": ["^sh$", "^sed$"]} # must exact match
self.regex_net = {
    "net address":["\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+", "\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", ":\d+"], 
    "NIC": "eth[0-9]*"}
self.regex_mem = {"Memory Address": "0x[0-9a-zA-Z]{8}"}
self.regex_other = {"permission":"Permission.*", "ID":["UID", "GID"]}

### 為 Regex 分類

In [2]:
obj = {
            "sed command": "bin/sed", 
            "startup": ["/etc/rc", "/etc/init.d/"],
            "process_info": "/proc/", 
            "sed temp file": "/etc/sed", 
            "selinx": "/selinux", 
            "sys": ["/sys/", "bin/", "lsb-release"], 
            "uname":"uname(.)($)",
            "dns": ["mtab", "nsswitch.conf", "resolv.conf", "/hosts"],
        }
obj.items()

dict_items([('sed command', 'bin/sed'), ('startup', ['/etc/rc', '/etc/init.d/']), ('process_info', '/proc/'), ('sed temp file', '/etc/sed'), ('selinx', '/selinux'), ('sys', ['/sys/', 'bin/', 'lsb-release']), ('uname', 'uname(.)($)'), ('dns', ['mtab', 'nsswitch.conf', 'resolv.conf', '/hosts'])])

In [8]:
r_type = {
    'file': ['bin/sed','/etc/rc', '/etc/init.d/','/proc/','/etc/sed','/selinux','/sys/', 'bin/', 'lsb-release','uname(.)($)','mtab', 'nsswitch.conf', 'resolv.conf', '/hosts'],
    'proc': ["^sh$", "^sed$"],
    'net': [r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}:\d+", r"\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", r'eth[0-9]$'],
    'mem': ["0x[0-9a-zA-Z]{8}"],
    'other': ["Permission.*","UID","GID"]
}
lookup_dic = {}
for k,v in r_type.items():
    for x in v:
        lookup_dic[x] = k

In [10]:
from collections import Counter
re_string_list = [
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['mtab', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'Permission.*'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', 'nsswitch.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', 'resolv.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', '/hosts', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', 'nsswitch.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', 'resolv.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', '/hosts', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'],
    ['mtab', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'Permission.*'],
]
type_list = []
for re_string in re_string_list:
    t = [lookup_dic[s] for s in re_string]
    type_list.append(Counter(t))

types = ['file', 'proc', 'net', 'mem', 'other']
for t in types:
    output = f'# of {t}_r'
    for sample in type_list:
        try:
            appear = sample[t]
        except:
            appear = 0
        output += f"{appear:3} "
    print(output)


# of file_r  8   8   8   8   8   3   8   8  11  11   8   8   3 
# of proc_r  2   2   2   2   2   0   2   2   2   2   2   2   0 
# of net_r  2   2   2   2   2   2   2   2   2   2   2   2   2 
# of mem_r  0   0   0   0   0   0   0   0   0   0   0   0   0 
# of other_r  1   1   1   1   1   1   1   1   1   1   1   1   1 


In [17]:
type_list

[Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'file': 3, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 11, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 11, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'proc': 2, 'file': 8, 'net': 2, 'other': 1}),
 Counter({'file': 3, 'net': 2, 'other': 1})]

In [19]:
S9 = {'^sh$', '/proc/', 'nsswitch.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', 'resolv.conf', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', '/hosts', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'}
S8 = {'^sh$', '/proc/', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}:\\d+', 'bin/', '/etc/rc', '/etc/sed', '\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}\\.\\d{1,3}', '/sys/', 'bin/sed', '^sed$', 'Permission.*', '/selinux', '/etc/init.d/'}

S9 - S8

{'/hosts', 'nsswitch.conf', 'resolv.conf'}

In [20]:
S8 - S9

set()

union 這些 regex

In [ ]:
re_string_list

In [9]:
from collections import Counter
master = RegexMaster()
all_regex_list = master.all_regex_list


t = [lookup_dic[s] for s in all_regex_list]
print(Counter(t))

NameError: name 'Counter' is not defined